In [1]:
import pandas as pd
import sys

sys.path.append("..")
from data_generation_scripts.utils import check_rate_limit, get_core_users_repos
from data_generation_scripts.generate_org_users_interactions import get_org_users_activities
from data_generation_scripts.generate_user_metadata import check_total_results

In [2]:
rates_df = check_rate_limit()

### Get Initial Organizations based on Search Queries

In [25]:
search_queries_user_df = pd.read_csv("../data/derived_files/updated_search_queries_user_join_subset_dh_dataset.csv")
initial_core_orgs = search_queries_user_df[search_queries_user_df['type'] == 'Organization']
initial_core_orgs = initial_core_orgs[initial_core_orgs['login'].notna()]
initial_core_orgs = initial_core_orgs.drop_duplicates(subset=['login'])
core_users, core_repos = get_core_users_repos()
if len(core_users) == 0:
    core_orgs = initial_core_orgs
else:
    core_orgs = core_users[core_users["type"] == "Organization"]
    if len(initial_core_orgs) != len(core_orgs):
        core_orgs = core_orgs[core_orgs.login.isin(initial_core_orgs.login)]
        initial_core_orgs = initial_core_orgs[~initial_core_orgs.login.isin(core_orgs.login.unique())]
        core_orgs = pd.concat([core_orgs, initial_core_orgs])


### Get Org Counts

In [26]:
org_count_cols = ["public_repos", "public_gists", "followers", "following", "star_count", "subscription_count","organization_count", "members_count"]

In [28]:
test = pd.read_csv("../data/metadata_files/user_url_cols.csv")
test

,col_name,col_url
0,followers,followers
1,following,following
2,public_repos,public_repos
3,public_gists,public_gists
4,star_count,starred_url
5,subscription_count,subscriptions_url
6,organization_count,organizations_url


In [27]:
for col in org_count_cols:
    if col in core_orgs.columns:
        needs_filling = core_orgs[core_orgs[col].isnull()]
        if len(needs_filling) > 0:
            print(col, len(needs_filling))
    else:
        print("missing entirely", col)

subscription_count 9
organization_count 9
missing entirely members_count


### Get Organization Members

In [4]:
### Add url to core_orgs
core_orgs["members_url"] = core_orgs["url"].apply(lambda x: x + "/public_members")

In [11]:
core_orgs.members_url = core_orgs.members_url.str.replace('users', 'orgs')

In [13]:
core_orgs = check_total_results(core_orgs, "members_count", "members_url")

Getting total results for each user:  56%|█████▌    | 236/421 [00:41<00:33,  5.56it/s]

response code: 404. hit rate limiting. trying to sleep...


Getting total results for each user:  56%|█████▋    | 237/421 [02:42<1:51:16, 36.29s/it]

query failed twice with code 404. Failing URL: https://api.github.com/orgs/dhandlib/public_members?per_page=1


Getting total results for each user:  75%|███████▍  | 315/421 [02:55<00:18,  5.85it/s]  

...

Getting total results for each user:  75%|███████▌  | 316/421 [04:56<1:03:28, 36.27s/it]

query failed twice with code 404. Failing URL: https://api.github.com/orgs/CDH-DevTeam/public_members?per_page=1


Getting total results for each user:  92%|█████████▏| 387/421 [05:08<00:06,  5.49it/s]  

response code: 404. hit rate limiting. trying to sleep...
query failed twice with code 404. Failing URL: https://api.github.com/orgs/HumanidadesDigitales/public_members?per_page=1


Getting total results for each user: 100%|██████████| 421/421 [07:15<00:00,  1.03s/it]


In [15]:
core_orgs.to_csv("../data/derived_files/core_orgs.csv", index=False)

In [16]:
core_orgs.members_count = core_orgs.members_count.astype(int)
core_orgs.members_count.sum()

727

In [ ]:
org_members_output_path = "../data/large_files/join_files/org_members_join_dataset.csv"
users_output_path = "../data/entity_files/users_dataset.csv"
get_url_field = "members_url"
load_existing_files = True
overwrite_existing_temp_files = False

users_followers_df, user_df = get_org_users_activities(core_orgs,org_members_output_path, users_output_path, get_url_field, load_existing_files, overwrite_existing_temp_files)